### Things to be improved in the future 

1) When using raw stats like this, need to think about different paces of play.   
2) Maybe isolate non-conference and conference games (add a feature to mark this)  
3) Develop and use an EPA or WPA model in some way   
4) Conference adjusted stats. 

In [1]:
import pandas as pd
import re

In [2]:
def NumberFromStrings(string):
    numberfromstring = re.findall('\d+',string)
    return numberfromstring

In [3]:
clean_games = pd.read_csv("data/final_data/team_games_data.csv")
clean_games['Team1'] = clean_games['Team1'].str.strip()
clean_games['year'] = clean_games['game_id'].apply(NumberFromStrings).apply(lambda x: x[0])
clean_games_18 = clean_games[clean_games['year'] == '2018'].drop('year',axis=1)
clean_games = clean_games.drop('year',axis=1)
clean_games = clean_games.convert_objects(convert_numeric=True)
clean_games_18 = clean_games_18.convert_objects(convert_numeric=True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  import sys


In [4]:
## Load bowl games

In [5]:
bowl_sched = pd.read_csv("data/bowl_sched.csv")

In [6]:
def avg_calcs_function(team_name,team2_cols=False):
    team_df_subset = clean_games_18[clean_games_18['Team1'] == team_name]
    if(len(team_df_subset)==0):
        team_df_subset = clean_games[clean_games['Team1'] == team_name]
    try:
        row = team_df_subset.sample(frac=0.4).drop(['game_id','Team1'],axis=1).mean()
    except: 
        team_df_subset = clean_games 
        row = team_df_subset.sample(frac=0.4).drop(['game_id','Team1'],axis=1).mean()
    if(team2_cols):
        t2_cols_rename = [word.replace('1','2') for word in row.index.tolist()]
        row.index = t2_cols_rename
    return(row)

In [7]:
import pickle
import utils
from sklearn.externals import joblib
scale = joblib.load("models/scaler.sav")
RF_model = pickle.load(open('models/RF_model.sav', 'rb'))
log_model = pickle.load(open('models/Log_model.sav','rb'))
svm_model = pickle.load(open('models/SVC_model.sav','rb'))

In [8]:
def one_time_pred(bowl_df,mod,scale):
    bowl_stats_team1 = bowl_df['Team1'].apply(avg_calcs_function)
    bowl_stats_team2 = bowl_df['Team2'].apply(avg_calcs_function,team2_cols=True)
    bowl_stats_df = pd.concat([bowl_stats_team1,bowl_stats_team2],axis=1)
    order_cols = ['Team1_first_downs', 'Team1_passing_first_downs',
       'Team1_rushing_first_downs', 'Team1_rushing_yds',
       'Team1_rushing_attempts', 'Team1_passing_attempts',
       'Team1_passing_completions', 'Team1_passing_interceptions',
       'Team1_avg_per_pass', 'Team1_pass_yds', 'Team1_yds_per_play',
       'Team1_tot_offense', 'Team1_fumbles_number', 'Team1_fumbles_lost',
       'Team1_pen_number', 'Team1_pen_yds', 'Team1_pun_number',
       'Team1_pun_yds', 'Team1_pun_re_number', 'Team1_pun_re_yds',
       'Team1_kick_re_number', 'Team1_kick_re_yds', 'Team1_int_re_yds',
       'Team1_third_down_attempts', 'Team1_third_down_suc',
       'Team1_fourth_down_attempts', 'Team1_fourth_down_suc',
       'Team2_first_downs', 'Team2_passing_first_downs',
       'Team2_rushing_first_downs', 'Team2_rushing_yds',
       'Team2_rushing_attempts', 'Team2_passing_attempts',
       'Team2_passing_completions', 'Team2_passing_interceptions',
       'Team2_avg_per_pass', 'Team2_pass_yds', 'Team2_yds_per_play',
       'Team2_tot_offense', 'Team2_fumbles_number', 'Team2_fumbles_lost',
       'Team2_pen_number', 'Team2_pen_yds', 'Team2_pun_number',
       'Team2_pun_yds', 'Team2_pun_re_number', 'Team2_pun_re_yds',
       'Team2_kick_re_number', 'Team2_kick_re_yds', 'Team2_int_re_yds',
       'Team2_third_down_attempts', 'Team2_third_down_suc',
       'Team2_fourth_down_attempts', 'Team2_fourth_down_suc',
       'Team1_third_down_conv_pct', 'Team2_third_down_conv_pct',
       'Team1_fourth_down_conv_pct', 'Team2_fourth_down_conv_pct',
       'Team1_turnover_margin', 'Team2_turnover_margin']
    bowl_dfs_clean = bowl_stats_df[order_cols]
    bowl_dfs_clean['Team2_win'] = 0
    X,y,c_names = utils.setup_data(bowl_dfs_clean,scale,'Team2_win',names=True)
    res = mod.predict(X)
    res = res.tolist()
    return(res)

In [23]:
def monte_carlo_preds(mod,n_sims=10):
    res = {}
    for i in range(n_sims):
        res[i] = one_time_pred(bowl_sched,mod,scale)
    res_df = pd.DataFrame(res).sum(axis=1)/n_sims
    return(res_df)

In [10]:
bowl_sched['Team2_win_RF']  = monte_carlo_preds(10000,RF_model,scale)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
bowl_sched.sort_values('Team2_win_RF',ascending=False)

,Date,Time,Bowl,Team1,Team2,Team2_win_RF
27,2018-12-22,3:30 PM,Armed Forces Bowl,Army,Houston,0.9999
29,2018-12-21,4:00 PM,Famous Idaho Potato Bowl,BYU,Western Mich.,0.9757
6,2018-12-31,7:00 PM,Holiday Bowl,Northwestern,Utah,0.9584
38,2018-12-15,1:30 PM,AutoNation Cure Bowl,Louisiana,Tulane,0.9370
35,2018-12-15,5:30 PM,Camellia Bowl,Eastern Mich.,Ga. Southern,0.9024
20,2018-12-27,5:15 PM,Pinstripe Bowl,Miami (Fla.),Wisconsin,0.9021
8,2018-12-31,3:00 PM,Redbox Bowl,Michigan State,Oregon,0.8777
5,2018-12-31,7:30 PM,Gator Bowl,NC State,Texas A&M,0.8717
16,2018-12-28,9:00 PM,Alamo Bowl,Iowa State,Washington St.,0.8688
31,2018-12-20,8:00 PM,Bad Boy Mowers Gasparilla Bowl,Marshall,South Florida,0.8430


In [28]:
log_model.n_jobs=-1

In [34]:
bowl_sched['Team2_win_log'] = monte_carlo_preds(log_model,n_sims=1000)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [35]:
bowl_sched['Team2_win_SVM'] = monte_carlo_preds(svm_model,n_sims=1000)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [36]:
bowl_sched.to_csv('data/final_data/predictions.csv')

In [38]:
import numpy as np

In [ ]:
np.abs

In [39]:
bowl_sched['conf_RF'] = np.abs(bowl_sched['Team2_win_RF'] - 0.5)
bowl_sched['conf_log'] = np.abs(bowl_sched['Team2_win_log']-0.5)
bowl_sched['conf_svm'] = np.abs(bowl_sched['Team2_win_SVM']-0.5)

In [44]:
bowl_sched.sort_values('conf_svm',ascending=False)

,Date,Time,Bowl,Team1,Team2,Team2_win_RF,Team2_win_log,Team2_win_SVM,conf_RF,conf_log,conf_svm
32,2018-12-19,8:00 PM,Frisco Bowl,Ohio,San Diego State,0.0044,0.000,0.001,0.4956,0.500,0.499
3,2019-01-01,1:00 PM,Fiesta Bowl,UCF,LSU,0.0162,0.016,0.011,0.4838,0.484,0.489
33,2018-12-18,7:00 PM,Boca Raton Bowl,UAB,Northern Ill.,0.0488,0.024,0.020,0.4512,0.476,0.480
0,2019-01-01,8:45 PM,Sugar Bowl,Georgia,Texas,0.0042,0.018,0.027,0.4958,0.482,0.473
17,2018-12-28,5:15 PM,Camping World Bowl,Syracuse,West Virginia,0.7873,0.955,0.968,0.2873,0.455,0.468
27,2018-12-22,3:30 PM,Armed Forces Bowl,Army,Houston,0.9999,0.985,0.964,0.4999,0.485,0.464
6,2018-12-31,7:00 PM,Holiday Bowl,Northwestern,Utah,0.9584,0.970,0.952,0.4584,0.470,0.452
35,2018-12-15,5:30 PM,Camellia Bowl,Eastern Mich.,Ga. Southern,0.9024,0.937,0.942,0.4024,0.437,0.442
11,2018-12-29,8:00 PM,Orange Bowl,Alabama,Oklahoma,0.5682,0.874,0.932,0.0682,0.374,0.432
21,2018-12-27,1:30 PM,Independence Bowl,Duke,Temple,0.8140,0.931,0.920,0.3140,0.431,0.420
